# Assistant API Demo
This notebook acts as an introduction into the assistants API and some of the potential use cases

In [29]:
import os
import json
from dotenv import find_dotenv,load_dotenv

from openai import OpenAI 

# Basic set-up

In [4]:
#loading enviroment variables 
load_dotenv(find_dotenv())

client = OpenAI(
  api_key=os.environ.get("OPENAI_API_KEY"),
)

In [5]:
# Making a simple chat request 
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

In [22]:
print(completion.choices[0].message.content[:184])

In the realm of code, where logic converges,
A concept arises, both puzzling and emergent,
A dance of repetition, elegant and profound,
Welcome, dear friend, to the world of recursion!


## Keeping context
Now that is quite simple but if we want it to keep context of the chat we have had one thing we need to do is pass more messages in

In [32]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."},
    {"role": "system", "content": "In the realm of code, where logic converges, A concept arises, both puzzling and emergent,A dance of repetition, elegant and profound,Welcome, dear friend, to the world of recursion!"},
    {"role": "user", "content": "Okay can you change it to be about encapsulation and do it in under 200 words?"}
  ]
)

In [33]:
print(completion.choices[0].message.content)

In the realm of code, where secrets lie,
Lies a concept hidden from mortal eye,
A potent spell, a mystical dream,
Welcome, dear friends, to encapsulation supreme!

It's a fort of data, carefully sealed,
Shielded from prying eyes, its truth concealed,
Inside the walls, methods whisper and sing,
Oblivious to the chaos that the outside may bring.

Like ancient scrolls in a cryptic tome,
Encapsulation shelters, keeps data at home,
It binds them tight, away from harm's spoil,
Guarding its essence, within a sacred coil.

So cherish this art, this enchanted domain,
Where data is sovereign and safeguards remain,
For in the embrace of encapsulation pure,
Lies the beauty of code, unspoiled and secure.


In [28]:
# Nice it understands the task and keeps the same style 
len(completion.choices[0].message.content.split(' '))

136

## New Assistant API 
Setting up a little more complex but allows us a lot more to play with

In [47]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


# Step 1: send the conversation and available functions to the model
messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)
response_message = response.choices[0].message
tool_calls = response_message.tool_calls
# Step 2: check if the model wanted to call a function
if tool_calls:
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
    }  # only one function in this example, but you can have multiple
    messages.append(response_message)  # extend conversation with assistant's reply
    # Step 4: send the info for each function call and function response to the model
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
    second_response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
    )  # get a new response from the model where it can see the function response

second_response.choices[0].message.content

"Currently, the weather in San Francisco is 72°F, in Tokyo it's 10°C, and in Paris it's 22°C."

# Breakdown

In [ ]:
tools = [
    {
        # We can also use some of the other tools like code interpreters or Dalle 
        "type": "function",
        # Defining the function itself 
        "function": {
            # The name of your function in yoru code base 
            "name": "get_current_weather",
            # A brief description so the bot knows when to try to call this function 
            "description": "Get the current weather in a given location",
            # Defining each parameter for the function 
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                # This allows us to make any of our parameteres required e.g. this won't run if it can't get it 
                "required": ["location"],
            },
        },
    }
]

In [ ]:
# Very similar to earlier as we can see with the addition of the tools and tool_choice params 
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)

In [ ]:
# If we had more functions we would add them in here and to the tools 
available_functions = {
    # Important that the text name you give the function matches in tools and here so gpt knows how to build it 
        "get_current_weather": get_current_weather,
    }  # only one function in this example, but you can have multiple

In [45]:
for tool_call in tool_calls:
    print(tool_call)

ChatCompletionMessageToolCall(id='call_ES62QHybGF2SM4leEuHqxkPe', function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), type='function')
ChatCompletionMessageToolCall(id='call_ZcMMAho458bGArUOqQodUgXf', function=Function(arguments='{"location": "Tokyo, Japan", "unit": "celsius"}', name='get_current_weather'), type='function')
ChatCompletionMessageToolCall(id='call_mpcMgJLTCe0uwUvrchw2U0vL', function=Function(arguments='{"location": "Paris, France", "unit": "celsius"}', name='get_current_weather'), type='function')


In [50]:
# Gets the name of the function you want to run and then the actual function 
function_name = tool_call.function.name
function_to_call = available_functions[function_name]
#Retreives the parameteres for the function 
function_args = json.loads(tool_call.function.arguments)
# Calls the function with the approriate params 
function_response = function_to_call(
    location=function_args.get("location"),
    unit=function_args.get("unit"),
)
function_args

{'location': 'Paris, France', 'unit': 'celsius'}

# Another example

In [52]:
import requests as r 

In [70]:
def get_current_price(ticker:str):
    """Given a companies ticker return the current stock price """
    data = r.get(f'https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey=QJ0BZyAOPg4M4iXiYCMpxTmQ340N5KdU').json()
    return "$" + str(data[0]['price'])

get_current_price('AAPL')

'$191.12'

In [75]:
def run_prompt(messages:list):
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_price",
                "description": "Get the current price of a stock",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {
                            "type": "string",
                            "description": "The ticker of a stock",
                        },
                        
                    },
                    "required": ["ticker"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_price": get_current_price,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                ticker=function_args.get("ticker"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
    
messages = [{"role": "user", "content": "What's is AAPL stock price?"}]
run_prompt(messages).choices[0].message.content

'The current stock price for AAPL (Apple Inc.) is $191.2.'

# Different message??

In [79]:
messages = [{"role": "user", "content": "What's is Metas stock price?"}]
run_prompt(messages).choices[0].message.content

'The current stock price for Meta (formerly Facebook) is $339.345.'

In [80]:
messages = [{"role": "user", "content": "What's is Facebooks stock price?"}]
run_prompt(messages).choices[0].message.content

"I'm sorry, but I don't have the current stock price for Facebook (FB). You can check the latest stock price on a financial news website or by using a stock market app."

### Change the function 
What happens if we change the function to something taht returns all the data about the stock?? Will it be able to tell us more or just do a dump with all the extra information we give it??

In [86]:
def get_stock_information(ticker:str):
    """Given a companies ticker return the current stock price """
    data = r.get(f'https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey=QJ0BZyAOPg4M4iXiYCMpxTmQ340N5KdU').json()
    return str(data[0])

def run_prompt(messages:list):
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_stock_information",
                "description": "Get a json object of all current information about a stock",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {
                            "type": "string",
                            "description": "The ticker of a stock",
                        },
                        
                    },
                    "required": ["ticker"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_stock_information": get_stock_information,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                ticker=function_args.get("ticker"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
    
messages = [{"role": "user", "content": "What's is AAPL stock price?"}]
run_prompt(messages).choices[0].message.content

'The current stock price for AAPL (Apple Inc.) is $191.26 USD.'

## Interesting
It seems to be able to parse through the information we give it, Lets ask about some of the other attributes it returns

In [87]:
messages = [{"role": "user", "content": "What's is link to the image of apples company?"}]
run_prompt(messages).choices[0].message.content

'You can find the image of Apple Inc. on this link: [Apple Inc. Image](https://financialmodelingprep.com/image-stock/AAPL.png)'

In [88]:
messages = [{"role": "user", "content": "Can you give me a description of apple?"}]
run_prompt(messages).choices[0].message.content

"Apple Inc. is a multinational technology company that designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers a wide range of products, including the iPhone, Mac, iPad, and various wearables and accessories. In addition to its hardware offerings, Apple also provides various services such as AppleCare support, cloud services, and operates platforms like the App Store and Apple Music. The company serves consumers, small and mid-sized businesses, and the education, enterprise, and government markets. Apple distributes its products through its retail and online stores, as well as through third-party cellular network carriers, wholesalers, retailers, and resellers. The company was incorporated in 1977 and is headquartered in Cupertino, California. Apple's CEO is Mr. Timothy D. Cook, and it has approximately 161,000 full-time employees."

In [90]:
messages = [{"role": "user", "content": "Who is the CEO of apple"}]
run_prompt(messages).choices[0].message.content

'The CEO of Apple is Mr. Timothy D. Cook.'

# Overall
Overall I think the assistants API is the future of any development in this field of building business inteligence tools, This allows for rapid development in line with current dev standards!!